In [7]:
import json
import argparse
import datetime
import json
import random
import time
from pathlib import Path
import os, sys
import numpy as np

import torch
from torch.utils.data import DataLoader, DistributedSampler

from util.get_param_dicts import get_param_dict
from util.logger import setup_logger
from util.slconfig import DictAction, SLConfig
from util.utils import ModelEma, BestMetricHolder
import util.misc as utils

import datasets
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch, test
from datasets.mc_sample import MCSampler

In [2]:
def get_args_parser():
    parser = argparse.ArgumentParser('Set transformer detector', add_help=False)
    parser.add_argument('--config_file', '-c', type=str, default='config/DINO/DINO_5scale.py', required=False)

    # dataset parameters
    parser.add_argument('--modelname', default='dino')
    parser.add_argument('--dataset_file', default='coco')
    parser.add_argument('--coco_path', type=str, default='../coco/')
    parser.add_argument('--coco_panoptic_path', type=str)
    parser.add_argument('--remove_difficult', action='store_true')
    parser.add_argument('--fix_size', action='store_true')
    parser.add_argument('--masks', action='store_true')

    # training parameters
    parser.add_argument('--output_dir', default='',
                        help='path where to save, empty for no saving')
    parser.add_argument('--note', default='',
                        help='add some notes to the experiment')
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')
    parser.add_argument('--seed', default=42, type=int)
    parser.add_argument('--resume', default='', help='resume from checkpoint')
    parser.add_argument('--pretrain_model_path', help='load from other checkpoint')
    parser.add_argument('--finetune_ignore', type=str, nargs='+')
    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')
    parser.add_argument('--eval', action='store_true')
    parser.add_argument('--num_workers', default=10, type=int)
    parser.add_argument('--test', action='store_true')
    parser.add_argument('--debug', action='store_true')
    parser.add_argument('--find_unused_params', action='store_true')

    parser.add_argument('--save_results', action='store_true')
    parser.add_argument('--save_log', action='store_true')

    # distributed training parameters
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')
    parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
    parser.add_argument('--rank', default=0, type=int,
                        help='number of distributed processes')
    
    return parser

parser = argparse.ArgumentParser('DETR training and evaluation script', parents=[get_args_parser()])
args = parser.parse_args([])

In [3]:
dataset_val = build_dataset(image_set='val', args=args)
sampler_val = MCSampler(dataset_val, 100)

data_aug_params: {
  "scales": [
    480,
    512,
    544,
    576,
    608,
    640,
    672,
    704,
    736,
    768,
    800
  ],
  "max_size": 1333,
  "scales2_resize": [
    400,
    500,
    600
  ],
  "scales2_crop": [
    384,
    600
  ]
}
loading annotations into memory...
Done (t=0.42s)
creating index...
index created!


In [4]:
data_loader_val = DataLoader(dataset_val, 1, sampler=sampler_val, drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)

In [9]:
dictionary = {3: {"a":3}, 1: {"a":4}}
with open("sample.json", "w") as outfile:
    json.dump(dictionary, outfile)

In [10]:
with open('sample.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)
 
print(json_object)
print(type(json_object))

{'3': {'a': 3}, '1': {'a': 4}}
<class 'dict'>


In [8]:
np.random.randint(0, high=10000, size=90)

array([3787, 7139, 3082, 5583, 5404, 5950, 8868, 6872, 6672, 3136, 5427,
       2131, 9432, 6930, 9491, 4187, 8110,  481, 3141, 7746, 3549, 4096,
       1767, 2126, 4834, 2409, 9729, 4254, 9513, 7544, 4352, 1513, 8175,
       2811, 2804, 4470, 2389, 4837, 5658,  907, 2484, 2403, 5460, 9269,
       4132, 4111, 9473, 5592, 7820,  803, 4586, 6444, 6787, 1748,  173,
       1124, 8080, 5474, 2869, 5744, 5410, 5907, 7729, 6235, 6274, 1314,
       9561, 1343, 3234,  374, 9414, 6446, 2103, 3120,    0,  733, 3938,
       4215, 2467, 6064, 5355, 4593, 6251, 5541, 5512, 4428,  990, 1723,
       8246, 2992])

# cifar10

In [20]:
from tensorflow.keras import datasets
data = datasets.cifar10.load_data()

2023-05-18 20:48:04.944146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 20:48:05.090638: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-18 20:48:05.757842: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-18 20:48:05.757924: W tensorflow/

In [22]:
(x_train, y_train), (x_test, y_test) = data

In [23]:
len(x_train)

50000

In [24]:
len(x_test), len(y_test)

(10000, 10000)

In [29]:
a = torch.randperm(100)
a

tensor([38, 26, 16, 63, 86,  9, 96, 99,  3, 90, 43, 65, 18, 72, 23, 91, 12, 22,
        88, 34, 82, 78, 50,  6, 77, 14, 44, 15, 32, 97, 98, 36, 68, 92, 69, 41,
        57, 11,  5, 62,  2, 13, 20, 67, 56, 81, 71, 55, 30, 76, 61, 89, 10, 84,
        94, 49, 42,  8, 24, 79, 74, 85, 75, 45, 80, 46, 58, 27, 53, 70, 35, 25,
        31, 64, 48, 28, 51, 52, 73,  0, 33, 47, 37, 17, 21, 93, 87,  7, 60, 40,
         1, 59, 95, 54, 39, 29, 66,  4, 83, 19])

In [30]:
b = a[:50]

In [31]:
b

tensor([38, 26, 16, 63, 86,  9, 96, 99,  3, 90, 43, 65, 18, 72, 23, 91, 12, 22,
        88, 34, 82, 78, 50,  6, 77, 14, 44, 15, 32, 97, 98, 36, 68, 92, 69, 41,
        57, 11,  5, 62,  2, 13, 20, 67, 56, 81, 71, 55, 30, 76])

In [32]:
iter(b)